## DIA 046: Implementacion de Caching Avanzado con Redis para Mejorar el Rendimiento

Objetivo Principal:

Reducir la latencia y la carga en la API utilizando Redis para almacenar en caché resultados de predicción.
Integración de Redis:

Se inicializa un cliente de Redis utilizando variables de entorno para la configuración.
Se calcula una clave única (hash MD5) a partir del contenido de la imagen recibida para identificar la solicitud.
Lógica de Caching en el Endpoint:

Antes de procesar la predicción, se verifica si ya existe un resultado almacenado en caché para la imagen.
Si se encuentra en caché, se retorna el resultado sin reprocesar la imagen.
Si no se encuentra, se procesa la imagen, se obtiene la predicción y se almacena el resultado en Redis con un tiempo de expiración definido.
Beneficios:

Reducción de Latencia: Responde más rápidamente a solicitudes repetidas.
Optimización de Recursos: Disminuye la carga computacional al evitar reprocesar imágenes idénticas.
Escalabilidad: Facilita el manejo de grandes volúmenes de solicitudes.
Código Completo (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
import hashlib
from datetime import datetime
from functools import wraps

import requests
import redis
import numpy as np
from PIL import Image

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
from flasgger import Swagger

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Configuración de Flasgger para documentación
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": "apispec_1",
            "route": "/apispec_1.json",
            "rule_filter": lambda rule: True,
            "model_filter": lambda tag: True,
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, config=swagger_config)

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Inicialización de Redis
redis_client = redis.Redis(
    host=os.getenv("REDIS_HOST", "localhost"),
    port=int(os.getenv("REDIS_PORT", 6379)),
    db=0
)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Nuevo Modelo: AuditLog (para trazabilidad)
# ---------------------------
class AuditLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    action = db.Column(db.String(120), nullable=False)
    details = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "action": self.action,
            "details": self.details,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Función para Registro de Auditoría
# ---------------------------
def log_audit_event(username, action, details=""):
    audit = AuditLog(username=username, action=action, details=details)
    db.session.add(audit)
    db.session.commit()
    logger.info(f"Audit log: {username} - {action} - {details}")

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    User Login
    ---
    tags:
      - Auth
    parameters:
      - in: body
        name: credentials
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: JWT token generado.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    access_token = create_access_token(identity=username)
    log_audit_event(username, "login", "Usuario inició sesión")
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: La aplicación está funcionando correctamente.
    """
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoint de Predicción con Caching Avanzado (versión v1 modificada)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    """
    Predict Endpoint - v1 con Caching
    ---
    tags:
      - Prediction v1
    consumes:
      - multipart/form-data
    parameters:
      - in: formData
        name: file
        type: file
        required: true
        description: Imagen para predecir el dígito.
    responses:
      200:
        description: Predicción realizada.
        schema:
          type: object
          properties:
            prediccion:
              type: integer
            probabilidad:
              type: number
            version:
              type: string
            cached:
              type: boolean
    """
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400

    # Leer el contenido del archivo para generar una clave de caché
    file_bytes = file.read()
    cache_key = hashlib.md5(file_bytes).hexdigest()

    # Verificar si existe resultado en caché
    cached_result = redis_client.get(cache_key)
    if cached_result:
        logger.info("Resultado obtenido de la caché para la clave: %s", cache_key)
        result = json.loads(cached_result)
        result['cached'] = True
        log_audit_event(get_jwt_identity(), "predict_v1", "Resultado obtenido de la caché")
        return jsonify(result), 200

    # Si no hay caché, procesar la imagen
    image = Image.open(io.BytesIO(file_bytes)).convert('L')
    image = image.resize((28, 28))
    image_array = np.array(image).astype('float32') / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    image_array = np.expand_dims(image_array, axis=-1)

    # Simulación de predicción: retorna dígito 5 con 90% de probabilidad
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    result['cached'] = False
    # Guardar el resultado en Redis con expiración de 10 minutos (600 segundos)
    redis_client.setex(cache_key, 600, json.dumps(result))
    logger.info("v1: Predicción realizada y almacenada en caché con clave: %s", cache_key)
    log_audit_event(get_jwt_identity(), "predict_v1", "Predicción procesada y cacheada")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')

# ---------------------------
# Otros Endpoints (v2, A/B, retraining, reportes, etc.) se mantienen sin cambios para este día
# ---------------------------
# (Se omiten para este ejemplo)

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explicación del Código
Configuración de Redis:

Se inicializa un cliente Redis utilizando variables de entorno para REDIS_HOST y REDIS_PORT.
Cálculo de la Clave de Caché:

Se genera un hash MD5 a partir del contenido del archivo (imagen) para crear una clave única.
Lógica de Caching en el Endpoint /api/v1/predict:

Se verifica si la clave ya existe en Redis:
Si existe, se retorna el resultado almacenado con el indicador cached establecido en True.
Si no existe, se procesa la imagen (simulación de predicción) y se guarda el resultado en Redis con un tiempo de expiración de 10 minutos.
Se registran eventos de auditoría y logs para facilitar el diagnóstico.
Protección y Seguridad:

El endpoint está protegido con JWT y tiene un límite de 100 solicitudes por día.
Se utiliza el sistema de auditoría para registrar las acciones críticas.
Ejecución de la Aplicación:

La aplicación se ejecuta con Flask-SocketIO, lo que permite soporte para WebSockets y otras funcionalidades.
